# [How to combine results from multiple retrievers](https://python.langchain.com/docs/how_to/ensemble_retriever/)

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(
    texts=doc_list_1,
    metadatas=[{"source": "doc_list_1"}]*len(doc_list_1)
    )

# Number of documents to return.
bm25_retriever.k =2

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

faiss_vectorstore = FAISS.from_texts(
    texts=doc_list_2,
    embedding=OpenAIEmbeddings(),
    metadatas=[{"source": "doc_list_2"}]*len(doc_list_2),
    )
vector_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})


In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.5, 0.5]
    )

In [ ]:
docs = ensemble_retriever.invoke("apples")
docs

## Runtime Configuration
We can also configure the individual retrievers at runtime using configurable fields.  
Below we update the "top-k" parameter for the FAISS retriever specifically:

In [ ]:
from langchain_core.runnables import ConfigurableField

vector_retriever = faiss_vectorstore.as_retriever(
    search_kwargs={"k": 2}
    ).configurable_fields(
        search_kwargs=ConfigurableField(
            id="search_kwargs_faiss",
            name="search_kwargs",
            description="Search arguments for the retriever.",
            )
        )

In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.5, 0.5]
    )

In [ ]:
config = {"configurable": {"search_kwargs_faiss": {"k": 1}}}
docs = ensemble_retriever.invoke("apples", config=config)
docs